In [2]:
def enough_money(symbol):
    try:
        latest_price = rs.stocks.get_latest_price(symbol)
        latest_price = float(latest_price[0])
        buy_power = float(rs.profiles.load_account_profile(info = 'portfolio_cash'))-25000 
        #must subtract this value by 25K to prevent using required principle for trading
        #buy_power = float(rs.profiles.load_account_profile(info = 'cash_available_for_withdrawal'))
        #for testing
        #buy_power = 20
        if buy_power > latest_price:
            print(buy_power, latest_price)
            return True
        else:
            print("not enough money ", buy_power, latest_price)
            return False
    except:
        print("something didn't work")
        return False

In [1]:
def place_order(symbol):
    latest_price = rs.stocks.get_latest_price(symbol)
    latest_price = float(latest_price[0])
    #can adjust the purchase price here
    purchase_price = latest_price*1
    prev_buy_power = float(rs.profiles.load_account_profile(info = 'portfolio_cash'))-25000
    acct_prev = rs.profiles.load_account_profile()
    #print(acct_prev)
    #places an order based on latest_price
    order_dict = rs.orders.order_buy_limit(symbol=symbol, quantity=1, limitPrice=purchase_price, timeInForce = 'gtc')
    order_id = order_dict['id']
    result = order_dict['state']
    acct_post = rs.profiles.load_account_profile()
    #print(acct_post)
    post_buy_power = float(rs.profiles.load_account_profile(info = 'portfolio_cash'))-25000
    #print(prev_buy_power==post_buy_power)
    #this confirms the order is filled but DOES NOT confirm the stock is PURCHASED
    while result != 'filled' and prev_buy_power==post_buy_power:
        #print(prev_buy_power, post_buy_power)
        #get_result = requests.get(endpoint, headers=headers).json()
        get_result = rs.orders.get_stock_order_info(order_id)
        result = get_result['state']
        post_buy_power = float(rs.profiles.load_account_profile(info = 'portfolio_cash'))
        time.sleep(1)
        print("in while loop waitin for purchase to be filled")
    #I need to confirm the purchase before setting the sell price
    time.sleep(5)
    set_sell_price(symbol, purchase_price=purchase_price) 
    

In [2]:
def set_sell_price(symbol, purchase_price):
    #aiming for .9% return for each trade
    print("setting sell price")
    sell_price = purchase_price * 1.007
    rs.order_sell_limit(symbol=symbol, quantity=1, limitPrice=sell_price, timeInForce='gtc')

In [3]:
def buy_stock(sorted_upward_trending_stocks):
    #iterate through stocks
    for index, row in sorted_upward_trending_stocks.iterrows():        
        #ensures I have enough CASH to purchase stock
        if enough_money(row["symbol"]):
            print("next stock for purchase: ", row["symbol"])
            #uncomment line below to pause before making purchase
            #txt = input("enter something")
            
            if place_order(row["symbol"]):
                print("purchased", (row["symbol"]))
                #remove symbol from dataframe
                return None
        else:
            print("nope", row["symbol"])